In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [6]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [7]:
# nn layers
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [8]:
# Initialization
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-0.6645,  0.6646,  0.7159,  ..., -0.3040, -0.8945, -0.7977],
        [-1.2045,  2.4545, -1.8073,  ..., -1.7139, -0.0451,  0.3864],
        [ 0.3414,  0.3114, -1.9218,  ..., -0.4525, -0.6849,  0.9663],
        ...,
        [ 1.3915, -0.4048,  0.2338,  ..., -0.8257, -0.8397,  1.9816],
        [ 1.0414,  0.2130, -0.0417,  ...,  1.7541, -0.6454, -0.0821],
        [-0.1162, -1.2692,  0.8201,  ..., -0.8303, -0.5022,  0.0583]],
       requires_grad=True)

In [9]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [10]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for x, y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        x = x.view(-1,28*28).to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(x)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
        
    print("Epoch: ", "%04d" % (epoch+1), "cost =", "{:.9f}".format(avg_cost))
    
print("Learning finished")

Epoch:  14d cost = 130.154266357
Epoch:  24d cost = 36.310234070
Epoch:  34d cost = 22.885091782
Epoch:  44d cost = 15.922073364
Epoch:  54d cost = 11.547133446
Epoch:  64d cost = 8.626523972
Epoch:  74d cost = 6.448483944
Epoch:  104d cost = 4.804335117
Epoch:  114d cost = 3.676800728
Epoch:  124d cost = 2.644905806
Epoch:  134d cost = 1.990795493
Epoch:  144d cost = 1.596786976
Epoch:  154d cost = 1.220444679
Epoch:  164d cost = 0.993460000
Epoch:  174d cost = 0.814860165
Learning finished


In [12]:
# test the model using test sets
with torch.no_grad():
    x_test = mnist_test.test_data.view(-1,28*28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction,1) == y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy: ", accuracy.item())
    
    # get one and predict
    r = random.randint(0, len(mnist_test)-1)
    x_single_data = mnist_test.test_data[r:r+1].view(-1,28*28).float().to(device)
    y_single_data = mnist_test.test_labels[r:r+1].to(device)
    
    print("Label: ", y_single_data.item())
    single_prediction = model(x_single_data)
    print("Prediction: ", torch.argmax(single_prediction,1).item())

Accuracy:  0.9470000267028809
Label:  9
Prediction:  9


/Users/eumhojun/opt/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/eumhojun/opt/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
